In [199]:
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
import string

In [200]:
dataPath = "C:\\Users\\pc\\OneDrive - Al Akhawayn University in Ifrane\\Desktop\\Spring 2022\\Deep Learning\\Project\\Phase 2 - Mid-Report\\spam.csv"

In [201]:
df = pd.read_csv(dataPath, encoding = "ISO-8859-1")

In [202]:
df.drop('Unnamed: 2', inplace=True, axis=1)
df.drop('Unnamed: 3', inplace=True, axis=1)
df.drop('Unnamed: 4', inplace=True, axis=1)

df.head(5)

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [203]:
df.shape

(5572, 2)

In [204]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\pc\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [205]:
df.groupby('v1').count()

,v2
v1,
ham,4825
spam,747


In [206]:
df.head(5)

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [207]:
df['v2'] = df['v2'].str.replace('\W+', ' ').str.replace('\s+', ' ').str.strip()

df['v2'] = df['v2'].str.lower()

df['v2'] = df['v2'].str.split()

C:\Users\pc\AppData\Local\Temp\ipykernel_16968\1105765197.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['v2'] = df['v2'].str.replace('\W+', ' ').str.replace('\s+', ' ').str.strip()


In [208]:
df.head(5)

,v1,v2
0,ham,"[go, until, jurong, point, crazy, available, o..."
1,ham,"[ok, lar, joking, wif, u, oni]"
2,spam,"[free, entry, in, 2, a, wkly, comp, to, win, f..."
3,ham,"[u, dun, say, so, early, hor, u, c, already, t..."
4,ham,"[nah, i, don, t, think, he, goes, to, usf, he,..."


In [209]:
train = df.sample(frac=0.8,random_state=1).reset_index(drop=True)

test = df.drop(train.index).reset_index(drop=True)

train = train.reset_index(drop=True)

In [210]:
vocab = list(set(train['v2'].sum()))
word_counts_per_sms = pd.DataFrame([
    [row[1].count(word) for word in vocab]
    for _, row in train.iterrows()], columns=vocab)
train = pd.concat([train.reset_index(), word_counts_per_sms], axis=1).iloc[:,1:]

In [211]:
train.head(5)

,v1,v2,somethin,08714712394,frndship,hearing,due,howard,farm,chop,...,wn,08714712412,ecstasy,ibh,fall,3g,373,oil,behalf,identifier
0,ham,"[convey, my, regards, to, him]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,"[û_, anyway, many, good, evenings, to, u, s]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ham,"[my, sort, code, is, and, acc, no, is, the, ba...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ham,"[sorry, i, din, lock, my, keypad]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,spam,"[hi, babe, its, chloe, how, r, u, i, was, smas...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [212]:
ProbSpam = train['v1'].value_counts()['spam'] / train.shape[0]

ProbHam = train['v1'].value_counts()['ham'] / train.shape[0]

NumSpam = train.loc[train['v1'] == 'spam', 'v2'].apply(len).sum()

NumHam = train.loc[train['v1'] == 'ham','v2'].apply(len).sum()

vocabSize = len(train.columns) - 3

NaN = 1

In [213]:
def spamProb(word):
    if word in train.columns:
        return (train.loc[train['v1'] == 'spam', word].sum() + NaN) / (NumSpam + NaN*vocabSize)
    else:
        return 1

In [214]:
def hamProb(word):
    if word in train.columns:
        return (train.loc[train['v1'] == 'ham', word].sum() + NaN) / (NumHam + NaN*vocabSize)
    else:
        return 1

In [215]:
def classify(message):
    
    Probspam = ProbSpam
    Probham = ProbHam
    
    for word in message:
        Probspam *= spamProb(word)
        Probham *= hamProb(word)
    if Probspam > Probham:
        return 'spam'
    elif Probham > Probspam:
        return 'ham'
    else:
        return 'needs human classification'

In [216]:
classify('Hey Bro')

'ham'